In [1]:
import torch
import scvi
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np

sc.set_figure_params(dpi=100, frameon=False, color_map='Reds', facecolor=None)
sc.logging.print_header()
assert(scvi.__version__=='0.16.3')

from gradients import run_expected_jacobian_scanvi, run_integrated_jacobian_scanvi

/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:2: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion
Global seed set to 0
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.22.4 scipy==1.9.1 pandas==1.4.4 scikit-learn==1.1.2 statsmodels==0.13.2 python-igraph==0.10.2 pynndescent==0.5.7


/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(


In [2]:
base_path = '/home/icb/yuge.ji/projects/feature-attribution-sc'

In [3]:
hlca_path = '/lustre/groups/ml01/workspace/hlca_lisa.sikkema_malte.luecken/HLCA_reproducibility/data/HLCA_core_h5ads/HLCA_v1_integration/HLCA_v1_scANVI_input.h5ad'
adata = sc.read(hlca_path)
adata

/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/layers' was written without encoding metadata.
  d[k] = read_elem(f[k])
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:584: OldFormatWarning: Element '/obs/__categories/sample' was written without encoding metadata.
  categories = read_elem(categories_dset)
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:587: OldFormatWarning: Element '/obs/sample' was written without encoding metadata.
  read_elem(dataset), categories, ordered=ordered
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:584: OldFormatWarning: Element '/obs/__categories/original_celltype_ann' was written without encoding metadata.
  categories = read_elem(categories_dset)
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packa

/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:584: OldFormatWarning: Element '/obs/__categories/scanvi_label' was written without encoding metadata.
  categories = read_elem(categories_dset)
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:587: OldFormatWarning: Element '/obs/scanvi_label' was written without encoding metadata.
  read_elem(dataset), categories, ordered=ordered
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/anatomical_region_ccf_score' was written without encoding metadata.
  return read_elem(dataset)
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:584: OldFormatWarning: Element '/obs/__categories/manual_ann' was written without encoding metadata.
  categories = read_elem(categories_dset)
/home/icb/yuge.ji/miniconda3/en

/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:584: OldFormatWarning: Element '/raw/var/__categories/gene_symbols' was written without encoding metadata.
  categories = read_elem(categories_dset)
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:587: OldFormatWarning: Element '/raw/var/gene_symbols' was written without encoding metadata.
  read_elem(dataset), categories, ordered=ordered
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/raw/var/n_cells' was written without encoding metadata.
  return read_elem(dataset)
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/raw/var/highly_variable' was written without encoding metadata.
  return read_elem(dataset)
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.1

AnnData object with n_obs × n_vars = 587218 × 2000
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'subject_ID', 'subject_ID_as_published', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'Processing_site', 'dataset', 'anatomical_region_level_1', 'anatomical_region_level_2', 'anatomical_region_level_3', 'anatomical_region_highest_res', 'age', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_ne

Optional: subset in dataset when getting 3D attribution matrices.

In [4]:
adata.obs['cell_type_dataset'] = adata.obs.scanvi_label.astype(str) + '-' + adata.obs.dataset.astype(str)
sampled_data = adata.obs.groupby('cell_type_dataset').apply(lambda x: x.sample(min(200, len(x)), random_state=42))
indices = sampled_data.index.get_level_values(1)
adata = adata[indices].copy()

In [5]:
model = scvi.model.SCANVI.load('/home/icb/yuge.ji/projects/HLCA_reproducibility/notebooks/3_atlas_extension/scanvi_model/', adata)
model

INFO     File /home/icb/yuge.ji/projects/HLCA_reproducibility/notebooks/3_atlas_extension/sca
         nvi_model/model.pt already downloaded                                               


/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/scvi/data/_utils.py:156: UserWarning: Category 18 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/scvi/data/_utils.py:156: UserWarning: Category 18 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(


ScanVI Model with the following params: 
unlabeled_category: unlabeled, n_hidden: 128, n_latent: 30, n_layers: 2, dropout_rate: 0.1, 
dispersion: gene, gene_likelihood: nb
Training status: Trained

In [6]:
adata.obs["dataset"].value_counts()

Krasnow_2020                 4273
Banovich_Kropski_2020        4173
Misharin_2021                3891
Nawijn_2021                  2802
Barbry_Leroy_2020            2718
Meyer_2019                   2661
Lafyatis_Rojas_2019_10Xv2    2600
Misharin_Budinger_2018       2068
Teichmann_Meyer_2019         2066
Jain_Misharin_2021_10Xv2     1720
Lafyatis_Rojas_2019_10Xv1    1286
Jain_Misharin_2021_10Xv1     1259
Seibold_2020_10Xv3            986
Seibold_2020_10Xv2            873
Name: dataset, dtype: int64

In [7]:
def record_attrib(avg_jac, exp_jac_ds, sum_obs):
    """Takes care of aggregating different values."""
    if sum_obs:
        if avg_jac is None:
            avg_jac = exp_jac_ds
        else:
            avg_jac += exp_jac_ds  

    else:  # get attribution per cell
        if avg_jac is None:
            avg_jac = [exp_jac_ds]
        else:
            avg_jac.append(exp_jac_ds)
    return avg_jac

In [8]:
avg_abs_exp_jac = None
avg_exp_jac = None

sum_obs = False

In [9]:
for ds in adata.obs["dataset"].unique():
    print(ds)
    
    indices = np.where(adata.obs["dataset"] == ds)[0]
    batch_size = min(2000, int(len(indices) / 11))
    print(batch_size)
    
    dl_base = model._make_data_loader(adata=adata, shuffle=False, indices=indices, batch_size=batch_size)
    dl_prime = model._make_data_loader(adata=adata, shuffle=True, indices=indices, batch_size=batch_size)
    
    # sum abs values
    exp_jac_ds = run_expected_jacobian_scanvi(model.module.classify, dl_base, dl_prime, apply_abs=True, sum_obs=sum_obs)
    avg_abs_exp_jac = record_attrib(avg_abs_exp_jac, exp_jac_ds, sum_obs)

    # just sum
    exp_jac_ds = run_expected_jacobian_scanvi(model.module.classify, dl_base, dl_prime, apply_abs=False, sum_obs=sum_obs)
    avg_exp_jac = record_attrib(avg_exp_jac, exp_jac_ds, sum_obs)

Banovich_Kropski_2020
379
Barbry_Leroy_2020
247
Krasnow_2020
388
Lafyatis_Rojas_2019_10Xv1
116
Lafyatis_Rojas_2019_10Xv2
236
Meyer_2019
241
Misharin_2021
353
Misharin_Budinger_2018
188
Teichmann_Meyer_2019
187
Jain_Misharin_2021_10Xv1
114
Jain_Misharin_2021_10Xv2
156
Nawijn_2021
254
Seibold_2020_10Xv2
79
Seibold_2020_10Xv3
89


In [10]:
avg_abs_integr_jac = None
avg_integr_jac = None

sum_obs = False

In [11]:
for ds in adata.obs["dataset"].unique():
    print(ds)
    
    indices = np.where(adata.obs["dataset"] == ds)[0]
    dl_base = model._make_data_loader(adata=adata, shuffle=False, indices=indices, batch_size=2000)
    
    # sum abs values
    integr_jac_ds = run_integrated_jacobian_scanvi(model.module.classify, dl_base, apply_abs=True, sum_obs=sum_obs)
    avg_abs_integr_jac = record_attrib(avg_abs_integr_jac, integr_jac_ds, sum_obs)
    
    # just sum
    integr_jac_ds = run_integrated_jacobian_scanvi(model.module.classify, dl_base, apply_abs=True, sum_obs=sum_obs)
    avg_integr_jac = record_attrib(avg_integr_jac, integr_jac_ds, sum_obs)

Banovich_Kropski_2020
Barbry_Leroy_2020
Krasnow_2020
Lafyatis_Rojas_2019_10Xv1
Lafyatis_Rojas_2019_10Xv2
Meyer_2019
Misharin_2021
Misharin_Budinger_2018
Teichmann_Meyer_2019
Jain_Misharin_2021_10Xv1
Jain_Misharin_2021_10Xv2
Nawijn_2021
Seibold_2020_10Xv2
Seibold_2020_10Xv3


If `sum_obs = True`

In [47]:
from scvi import REGISTRY_KEYS
labels = model.adata_manager.get_state_registry(REGISTRY_KEYS.LABELS_KEY)['categorical_mapping']

In [49]:
pd.DataFrame(avg_abs_exp_jac.numpy(), columns=labels[:-1], index=adata.var_names.tolist()).to_csv("absolute_sum_expected_grads.csv", index_label="Gene")

In [50]:
pd.DataFrame(avg_exp_jac.numpy(), columns=labels[:-1], index=adata.var_names.tolist()).to_csv("sum_expected_grads.csv", index_label="Gene")

In [51]:
pd.DataFrame(avg_abs_integr_jac.numpy(), columns=labels[:-1], index=adata.var_names.tolist()).to_csv("absolute_sum_integrated_grads.csv", index_label="Gene")

In [52]:
pd.DataFrame(avg_integr_jac.numpy(), columns=labels[:-1], index=adata.var_names.tolist()).to_csv("sum_integrated_grads.csv", index_label="Gene")

If `sum_obs = False`

In [12]:
torch.cat(avg_exp_jac).shape

torch.Size([33376, 2000, 28])

In [14]:
np.save(
    '../../scripts/outputs/attribution_map_genes_3d_task2_expected_grads.npy',
    torch.cat(avg_exp_jac).cpu().detach().numpy().transpose(0, 2, 1)
)
np.save(
    '../../scripts/outputs/attribution_map_genes_3d_task2_integrated_grads.npy',
    torch.cat(avg_integr_jac).cpu().detach().numpy().transpose(0, 2, 1)
)